In [1]:
import pickle 
import numpy as np
import random
import dask
import dask.bag as db
import functools
import yaml

from dask_kubernetes import KubeCluster
from joblib import Memory
from dask.distributed import Client
from calculator_upload_empty import predict_E
from ocpmodels.models.dimenet_plus_plus import DimeNetPlusPlusWrap

import os

from enumeration_helper_script import enumerate_surface_wrap
from tqdm import tqdm
from ocdata.adsorbates import Adsorbate
from ocdata.bulk_obj import Bulk
from ocdata.base_atoms.pkls import BULK_PKL, ADSORBATE_PKL


amp: false
cmd:
  checkpoint_dir: /home/jovyan/ocp/checkpoints/2021-09-17-18-42-08
  commit: f4a0f08
  identifier: ''
  logs_dir: /home/jovyan/ocp/logs/tensorboard/2021-09-17-18-42-08
  print_every: 100
  results_dir: /home/jovyan/ocp/results/2021-09-17-18-42-08
  seed: null
  timestamp_id: 2021-09-17-18-42-08
dataset:
  grad_target_mean: 0.0
  grad_target_std: 2.887317180633545
  normalize_labels: true
  src: /home/jovyan/ocp/data/OC20/s2ef/all/test_id/
  target_mean: -0.7554450631141663
  target_std: 2.887317180633545
gpus: 0
logger: tensorboard
model: gemnet_t
model_attributes:
  activation: silu
  cbf:
    name: spherical_harmonics
  cutoff: 6.0
  direct_forces: true
  emb_size_atom: 512
  emb_size_bil_trip: 64
  emb_size_cbf: 16
  emb_size_edge: 512
  emb_size_rbf: 16
  emb_size_trip: 64
  envelope:
    exponent: 5
    name: polynomial
  extensive: true
  max_neighbors: 50
  num_after_skip: 2
  num_atom: 3
  num_before_skip: 1
  num_blocks: 3
  num_concat: 1
  num_radial: 128
  nu

In [3]:
# Initiate some helpful accessories
#------------------------------------------------------------------------------
name_to_num_dict = {1: 'H', 2: 'He', 3: 'Li', 4: 'Be', 5: 'B', 6: 'C', 7: 'N',
                    8: 'O', 9: 'F', 10: 'Ne', 11: 'Na', 12: 'Mg', 13: 'Al', 14: 'Si',
                    15: 'P', 16: 'S', 17: 'Cl', 18: 'Ar', 19: 'K', 20: 'Ca', 21: 'Sc',
                    22: 'Ti', 23: 'V', 24: 'Cr', 25: 'Mn', 26: 'Fe', 27: 'Co', 28: 'Ni',
                    29: 'Cu', 30: 'Zn', 31: 'Ga', 32: 'Ge', 33: 'As', 34: 'Se', 35: 'Br', 
                    36: 'Kr', 37: 'Rb', 38: 'Sr', 39: 'Y', 40: 'Zr', 41: 'Nb', 42: 'Mo',
                    43: 'Tc', 44: 'Ru', 45: 'Rh', 46: 'Pd', 47: 'Ag', 48: 'Cd', 49: 'In',
                    50: 'Sn', 51: 'Sb', 52: 'Te', 53: 'I', 54: 'Xe', 55: 'Cs', 56: 'Ba', 
                    57: 'La', 58: 'Ce', 59: 'Pr', 60: 'Nd', 61: 'Pm', 62: 'Sm', 63: 'Eu', 
                    64: 'Gd', 65: 'Tb', 66: 'Dy', 67: 'Ho', 68: 'Er', 69: 'Tm', 70: 'Yb',
                    71: 'Lu', 72: 'Hf', 73: 'Ta', 74: 'W', 75: 'Re', 76: 'Os', 77: 'Ir', 
                    78: 'Pt', 79: 'Au', 80: 'Hg', 81: 'Tl', 82: 'Pb', 83: 'Bi', 84: 'Po',
                    85: 'At', 86: 'Rn', 87: 'Fr', 88: 'Ra', 89: 'Ac', 90: 'Th', 91: 'Pa',
                    92: 'U', 93: 'Np', 94: 'Pu', 95: 'Am', 96: 'Cm', 97: 'Bk', 98: 'Cf',
                    99: 'Es', 100: 'Fm', 101: 'Md', 102: 'No', 103: 'Lr', 104: 'Rf',
                    105: 'Db', 106: 'Sg', 107: 'Bh', 108: 'Hs', 109: 'Mt', 110: 'Ds', 
                    111: 'Rg', 112: 'Cn', 113: 'Nh', 114: 'Fl', 115: 'Mc', 116: 'Lv',
                    117: 'Ts', 118: 'Og'}

transition_metal1 =  list(range(21,31))
transition_metal2 = list(range(39, 49))
transition_metal3 = list(range(72, 80))
transition_metals = [*transition_metal1, *transition_metal2, *transition_metal3]
# you can set elements_to_include = transition_metals
# ----------------------------------------------------------------------------





# INPUTS!!
# ----------------------------------------------------------------------------
# Bulk inputs
direct = False # boolean True = direct method, False = relaxations
mpids_to_slabs = True # True = use mpid_list to generate aslabs, False = use element list and num_of_els
elements_to_include = [29]# list of desired atomic numbers
mpid_list = ['mp-101']#, 'mp-2', 'mp-30', 'mp-74', 'mp-126', 'mp-101', 'mp-23', 'mp-124', 'mp-102']
num_of_els = [1] # [1,2] = unary and binary
mpids_to_exclude = ['mp-672234', 'mp-632250', 'mp-754514', 'mp-570747', 'mp-12103', 'mp-25', 
                    'mp-672233', 'mp-568584', 'mp-154', 'mp-999498', 'mp-14', 'mp-96', 'mp-1080711', 
                    'mp-1008394', 'mp-22848', 'mp-160', 'mp-1198724'] #these are odd and cause failures

# Adsorbate inputs
adsorbates_smile_list = ['*OH']

# File paths
worker_spec_path = '/home/jovyan/shared-scratch/Brook/worker-spec-relax.yml'
your_shared_scratch_path = '/home/jovyan/shared-scratch/Brook' # this will be used to create a cache dir in your folder

# Other
num_workers = 50 # should be scaled with workload. Use 1 to troubleshoot. 10 is a good place to start
max_size = 1000 # This is the maximum number of atoms you will allow in your slabs. Any greater will return size = Number rather than a prediction.
# -----------------------------------------------------------------------------





# Pre-work / organization
# -----------------------------------------------------------------------------
# convert el nums to names
elnames_to_include = [name_to_num_dict[el] for el in elements_to_include] 

# -----------------------------------------------------------------------------


In [4]:
# Create the workers
# -----------------------------------------------------------------------------
# Set the up a kube dask cluster 
cluster = KubeCluster(worker_spec_path, deploy_mode='local')
cluster.adapt(minimum=1, maximum = num_workers, interval = '30000 ms')
client = Client(cluster)

def _worker_upload(dask_worker, *, data, fname):
    dask_worker.loop.add_callback(
    callback=dask_worker.upload_file,
    comm=None,  # not used
    filename=fname,
    data=data,
    load=True)

## Upload code to every worker as they start/restart
fname_ = ['enumeration_helper_script.py', 'calculator_upload_empty.py']
for fname in fname_:
    with open(fname, 'rb') as f:
        data = f.read()
        client.register_worker_callbacks(
            setup=functools.partial(
                _worker_upload, data=data, fname=fname,
            )
        )

# Set up the cache directory - this will also be the local directory on each worker that will store cached results
location_preds = your_shared_scratch_path + '/cachedir_predictions_AIChE1'
location_slabs = your_shared_scratch_path + '/cachedir_slabs_AIChEpost'
memory_preds = Memory(location_preds,verbose=1)
memory_slabs = Memory(location_slabs,verbose=1)

distributed.http.proxy - INFO - To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
/opt/conda/lib/python3.8/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44111 instead
  warnings.warn(
distributed.scheduler - INFO - Clear task state
distributed.scheduler - INFO -   Scheduler at:   tcp://10.42.7.156:36903
distributed.scheduler - INFO -   dashboard at:                    :44111
distributed.deploy.adaptive - INFO - Adaptive scaling started: minimum=1 maximum=50
distributed.scheduler - INFO - Receive client connection: Client-32ddca5a-17e7-11ec-834f-eae5c63ff668
distributed.core - INFO - Starting established connection


In [5]:
# Open pkl of bulks and their associated info
with open(BULK_PKL, 'rb') as f:
    inv_index = pickle.load(f)
bulks = []

# Grab the desired bulk atoms object
if mpids_to_slabs:
    num_of_els = [1,2,3]
    for k in num_of_els:
        for itm in inv_index[k]:
            if itm[1] not in mpids_to_exclude and itm[1] in mpid_list:
                 bulks.append(itm)
else:
    for k in num_of_els:
        for itm in inv_index[k]: 
            els = itm[0].get_chemical_symbols() # Grab the chemical symbol of all
            els = np.unique(els)
            if itm[1] not in mpids_to_exclude and set(els).issubset(elnames_to_include):
                bulks.append(itm)

adsorbates_smile_list = ['*C']

# Open pkl of adsorbates and grab info for the ones in the smile list         
with open(ADSORBATE_PKL, 'rb') as f:
    inv_index = pickle.load(f)
ads_idx = [key for key in inv_index if inv_index[key][1] in adsorbates_smile_list]
adsorbates_obj = [Adsorbate(ADSORBATE_PKL, specified_index = idx) for idx in ads_idx]


#create a dask bag of adsorbates
adsorbate_bag = db.from_sequence(adsorbates_obj)

#create a dask bag of bulks
bulk_bag = db.from_sequence(bulks)

In [ ]:
# generate the adslab mappings
surfaces_bag = bulk_bag.map(memory_slabs.cache(enumerate_surface_wrap)).flatten()
surface_ads_combos = surfaces_bag.product(adsorbate_bag).repartition(npartitions=num_workers*20).compute()

#Load surfaces into dask bag
surfaces = db.from_sequence(surface_ads_combos)
# generate prediction mappings
predictions_bag = surfaces.map(memory_preds.cache(predict_E), max_size, direct)
# execute operations (go to all work)
predictions = predictions_bag.compute() # change to .compute() to push to local RAM


distributed.scheduler - INFO - Register worker <WorkerState 'tcp://10.42.0.133:40583', name: 0, memory: 0, processing: 2>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.42.0.133:40583
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <WorkerState 'tcp://10.42.0.134:43509', name: 3, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.42.0.134:43509
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <WorkerState 'tcp://10.42.0.135:38787', name: 4, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.42.0.135:38787
distributed.core - INFO - Starting established connection
distributed.scheduler - INFO - Register worker <WorkerState 'tcp://10.42.18.64:35715', name: 5, memory: 0, processing: 0>
distributed.scheduler - INFO - Starting worker compute stream, tcp://10.42.18